In [1]:
from aaff import aaff
from FcMole import FcM
from alch_deriv import alch_deriv
from pyscf import gto,scf
import numpy as np 
import matplotlib.pyplot as plt
from pyscf.grad import rhf as grhf
import basis_set_exchange as bse

In [2]:
mol=gto.M(atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))

In [3]:
hfCO=scf.RHF(mol)
hfCO.scf()

converged SCF energy = -112.786616218695


-112.78661621869524

In [4]:
U,dP,e1=alch_deriv(hfCO,[[0,1],[1,-1]])

In [5]:
hfCO.Gradients().grad()

--------------- RHF gradients ---------------
         x                y                z
0 C     0.0000000000     0.0000000000    -0.0000019315
1 O    -0.0000000000    -0.0000000000     0.0000019315
----------------------------------------------


array([[ 5.89707777e-15,  9.17133257e-15, -1.93146142e-06],
       [-5.89707777e-15, -9.17133257e-15,  1.93146284e-06]])

In [6]:
hfCO.Gradients().grad_elec()

array([[ 5.89707777e-15,  9.17133257e-15, -1.10657165e+01],
       [-5.89707777e-15, -9.17133257e-15,  1.10657165e+01]])

In [7]:
from FDcoeffs import *

In [8]:
dl=.5
gs=[]
ge=[]
amds=[]
for i in range(-4,5):
    fm=FcM(fcs=[i*dl,-i*dl],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]),verbose=0)
    mf=scf.RHF(fm)
    mf.scf(dm0=mf.init_guess_by_1e(),max_cycle=100)
    g=mf.Gradients()
    gs.append(g.grad())
    ge.append(g.grad_elec())
    amds.append(aaff(mf,[1,-1]))

In [9]:
ge[4]

array([[ 1.67047064e-14, -4.31135987e-14, -1.10657172e+01],
       [-1.67047064e-14,  4.31135987e-14,  1.10657172e+01]])

In [10]:
#forward difference 
(ge[5]-ge[4])/dl

array([[-4.07737714e-14,  8.82114389e-14, -5.23677703e-01],
       [ 4.07737714e-14, -8.82114389e-14,  5.23677703e-01]])

In [11]:
print("2 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_2,(np.asarray(ge)))/dl)
print("4 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_4,(np.asarray(ge)))/dl)
print("6 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_6,(np.asarray(ge)))/dl)
print("8 order finite diff. accuracy:: \n",np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))/dl)

2 order finite diff. accuracy:: 
 [[ 6.07205910e-15  8.39515081e-15 -6.77666144e-01]
 [-6.07205910e-15 -8.39515081e-15  6.77666144e-01]]
4 order finite diff. accuracy:: 
 [[ 2.22176703e-14  1.62430185e-14 -6.85183479e-01]
 [-2.22176703e-14 -1.62430185e-14  6.85183479e-01]]
6 order finite diff. accuracy:: 
 [[ 3.51218306e-14  2.17886678e-14 -6.85575233e-01]
 [-3.51218306e-14 -2.17886678e-14  6.85575233e-01]]
8 order finite diff. accuracy:: 
 [[ 4.48960021e-14  2.57104508e-14 -6.85618026e-01]
 [-4.48960021e-14 -2.57104508e-14  6.85618026e-01]]


In [12]:
aaff(hfCO,[1,-1])

array([[ 1.54851247e-14,  3.11951868e-16, -6.85627204e-01],
       [-1.54851247e-14, -3.11951868e-16,  6.85627204e-01]])

In [13]:
print(np.einsum('i,ijk->jk',fd2_8,(np.asarray(ge)))/dl**2)
print(np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))/dl)

[[-3.44687667e-13  4.26577662e-13  6.19053615e-01]
 [ 3.44687667e-13 -4.26577662e-13 -6.19053615e-01]]
[[-1.80896260e-14  2.67977413e-14  6.18863830e-01]
 [ 1.80896260e-14 -2.67977413e-14 -6.18863830e-01]]


In [14]:
print(np.einsum('i,ijk->jk',fd3_6,(np.asarray(ge)))/dl**3)
print(np.einsum('i,ijk->jk',fd2_6,(np.asarray(amds)))/dl**2)

[[-1.09390908e-12 -4.82829883e-13  1.93566569e-01]
 [ 1.09390908e-12  4.82829883e-13 -1.93566569e-01]]
[[ 9.21753491e-13 -1.48760154e-12  1.93908091e-01]
 [-9.21753491e-13  1.48760154e-12 -1.93908091e-01]]


In [15]:
print(np.einsum('i,ijk->jk',fd4_2,(np.asarray(ge)))/dl**4)
print(np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))/dl**3)

[[ 3.82829434e-12 -3.22350906e-12 -1.36370530e-01]
 [-3.82829434e-12  3.22350906e-12  1.36370530e-01]]
[[ 4.61756975e-13 -2.13963162e-13 -1.47621861e-01]
 [-4.61756975e-13  2.13963162e-13  1.47621861e-01]]


In [16]:
print(np.einsum('i,ijk->jk',fd6_4,(np.asarray(ge)))/dl**6)
print(np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))/dl**5)
#print(np.einsum('i,ijk->jk',fd6_4,(np.asarray(amds)))/dl**6)

[[-1.36389136e-10  6.88164685e-11  4.21289969e-01]
 [ 1.36389136e-10 -6.88164685e-11 -4.21289969e-01]]
[[-7.57277783e-12  9.48011963e-13  3.43309838e-01]
 [ 7.57277783e-12 -9.48011963e-13 -3.43309838e-01]]


In [17]:
#with dl=.1  the 4th derivatives are:   only McL expansion of aaff is consistent to obtain APDFT4_g1 derivs
#[[-5.43974711e-09 -4.46784523e-09 -8.97038551e-02]
# [ 5.43974711e-09  4.46784523e-09  8.97038538e-02]]
#[[ 1.04025962e-09  1.59691820e-09 -1.55567761e-01]
# [-2.70775954e-08  7.58253933e-09  1.56759171e-01]]

In [18]:
NN=FcM(fcs=[1,-1],atom="C 0 0 0; O 0 0 2.08272",unit="Bohrs",basis=bse.get_basis("pcX-2",fmt="nwchem",elements=[5,6,7,8,9]))
hfNN=scf.RHF(NN)
hfNN.scf(dm0=hfNN.init_guess_by_1e())
gNN=hfNN.Gradients().grad_elec()

converged SCF energy = -108.984680830552


In [19]:
gNN,hfNN.Gradients().grad_nuc()+gNN

(array([[-1.55566798e-15, -2.70308700e-15, -1.14171161e+01],
        [ 1.55566798e-15,  2.70308700e-15,  1.14171161e+01]]),
 array([[-1.55566798e-15, -2.70308700e-15, -1.20865800e-01],
        [ 1.55566798e-15,  2.70308700e-15,  1.20865800e-01]]))

In [20]:
hfNN.Gradients().grad_nuc()+ge[4]+amds[4]+np.einsum('i,ijk->jk',fd1_2,(np.asarray(amds)))/dl/2  \
    +np.einsum('i,ijk->jk',fd2_2,(np.asarray(amds)))/dl**2/6+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))/dl**3/24 \
  +np.einsum('i,ijk->jk',fd4_4,(np.asarray(amds)))/dl**4/120+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))/dl**5/720 \
 +np.einsum('i,ijk->jk',fd6_2,(np.asarray(amds)))/dl**6/5040

array([[-3.63735127e-14,  5.79486194e-14, -1.24588144e-01],
       [ 3.63735127e-14, -5.79486194e-14,  1.24588144e-01]])

In [21]:
hfNN.Gradients().grad_nuc()+ge[4]+amds[4]/2,amds[4]

(array([[-2.72881205e-14,  2.75448411e-14, -1.12280522e-01],
        [ 2.72881205e-14, -2.75448411e-14,  1.12280522e-01]]),
 array([[-8.79856538e-14,  1.41316880e-13, -6.85627119e-01],
        [ 8.79856538e-14, -1.41316880e-13,  6.85627119e-01]]))

In [22]:
(hfNN.Gradients().grad_nuc()-hfCO.Gradients().grad_nuc())/(-2)

array([[-0.        , -0.        , -0.11526786],
       [-0.        , -0.        ,  0.11526786]])

In [23]:
aaff_predict=[]
aaff_predict.append((hfNN.Gradients().grad_nuc()+ge[4])[1,2])
aaff_predict.append(aaff_predict[0]+amds[4][1,2])
aaff_predict.append(aaff_predict[1]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(amds)))[1,2]/dl/2)
aaff_predict.append(aaff_predict[2]+np.einsum('i,ijk->jk',fd2_6,(np.asarray(amds)))[1,2]/dl**2/6)
aaff_predict.append(aaff_predict[3]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(amds)))[1,2]/dl**3/24 )
aaff_predict.append(aaff_predict[4]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(amds)))[1,2]/dl**4/120)
aaff_predict.append(aaff_predict[5]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(amds)))[1,2]/dl**5/720)


In [24]:
aaff_predict

[-0.2305330369892573,
 0.45509408197938095,
 0.14566216699844475,
 0.1133441518935063,
 0.11949506276031703,
 0.12139314501291774,
 0.1209163257931425]

In [25]:
fd_predict=[]
fd_predict.append((hfNN.Gradients().grad_nuc()+ge[4])[1,2])
fd_predict.append(fd_predict[0]+np.einsum('i,ijk->jk',fd1_8,(np.asarray(ge)))[1,2]/dl)
fd_predict.append(fd_predict[1]+np.einsum('i,ijk->jk',fd2_6,(np.asarray(ge)))[1,2]/dl**2/2)
fd_predict.append(fd_predict[2]+np.einsum('i,ijk->jk',fd3_6,(np.asarray(ge)))[1,2]/dl**3/6)
fd_predict.append(fd_predict[3]+np.einsum('i,ijk->jk',fd4_4,(np.asarray(ge)))[1,2]/dl**4/24 )
fd_predict.append(fd_predict[4]+np.einsum('i,ijk->jk',fd5_4,(np.asarray(ge)))[1,2]/dl**5/120)
fd_predict.append(fd_predict[5]+np.einsum('i,ijk->jk',fd6_2,(np.asarray(ge)))[1,2]/dl**6/720)
fd_predict

[-0.2305330369892573,
 0.45508498892408167,
 0.14558856794507785,
 0.11332747303076171,
 0.11950464531179984,
 0.12133793045596934,
 0.12094187695191691]

In [26]:
# trget 1.20865800e-01